In [ ]:
import sys

sys.path.insert(0, '../03_Remake_with_TDD/algorithms')
from ht_time import *

In [ ]:
ht_time = HattrickTime()
folder = ht_time.findLastSunOrWed()
# folder = '2018/04/08'
print("folder =", folder)

In [ ]:
import re
from bs4 import BeautifulSoup

In [ ]:
import sys

sys.path.insert(0, '../97_Parse/')
from parseMatch import *

sys.path.insert(0, '../98_Crawler/01_DualMonitor/')
from DualMonitor import getIsDualMonitor

sys.path.insert(0, '../98_Crawler/02_WebBrowser/')
from FirefoxTab import clickTabOfFireFox
from crawMatch import *

sys.path.insert(0, '../98_Crawler/03_OnScreen/')
from Empty import clickEmpty
from MoveScroll import moveScroll
from MoveFindMoveClick import moveByImage, moveByImageWithModify

sys.path.insert(0, '../98_Crawler/04_Login/')
from HattrickLogInOut import clickHattrickLogin, clickHattrickLogout

In [ ]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

clickTabOfFireFox(4, isDualMonitor)

clickEmpty(305, isDualMonitor)
moveScroll(20)
clickHattrickLogin(500, 305, isDualMonitor)

# clickHattrickLogout(1270, 370, isDualMonitor)

In [ ]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

# Hattrick HeavyDefence Tab
clickTabOfFireFox(4, isDualMonitor)

# # Hattrick Login by Facebook
clickEmpty(305, isDualMonitor)
moveScroll(20)
clickHattrickLogin(500, 305, isDualMonitor)

# Hattrick MyClub
moveScroll(20)
moveByImage(530, 342, 77, 42, isDualMonitor, 'screen-MyClub.png')

# Hattrick Match
clickEmpty(700, isDualMonitor)
moveScroll(20)
moveByImage(384, 698, 80, 108, isDualMonitor, 'screen-Match.png')

# Hattrick LastMatch
clickEmpty(831, isDualMonitor)
moveScroll(20)
# moveByImageWithModify(1040, 831, 15, -15, 58, 48, isDualMonitor, 'screen-LastMatch.png')
moveAndClick(1082, 841, isDualMonitor)
time.sleep(5)

# FirefoxFile
moveAndClick(90, 15, isDualMonitor)

# SavePageAs
time.sleep(3)
moveAndClick(91, 130, isDualMonitor)

# Type match
time.sleep(3)
typeFileName()

# rwoo folder
moveAndClick(570, 460, isDualMonitor)

# 02_workspace folder
moveAndClicks(720, 440, isDualMonitor, 2)

# 04_Hattrick folder
moveAndClicks(720, 500, isDualMonitor, 2)

# HattrickStudy
moveAndClicks(720, 415, isDualMonitor, 2)

# 01_Heavy2Defence
moveAndClicks(720, 440, isDualMonitor, 2)

# 2018
moveAndClicks(720, 440, isDualMonitor, 2)

# Last Month
clickLastMonthOrDay()

# Last Day
clickLastMonthOrDay()

# Save
moveByImage(1280, 847, 91, 41, isDualMonitor, 'screen-Save.png')

# Hattrick Logout by Facebook
clickEmpty(336, isDualMonitor)
moveScroll(20)
clickHattrickLogout(1270, 370, isDualMonitor)

# Jupyter Tab
clickTabOfFireFox(7, isDualMonitor)

In [ ]:
from ht_file import *

In [ ]:
HattrickFile = HattrickFile()
HattrickFile.check_and_delete_folder(folder + '/match_files')

In [ ]:
from ht_match import *

In [ ]:
HattrickMatch = HattrickMatch()
match_dict_list = HattrickMatch.findMatchList(folder + '/match.html', 'mydatabase2')
match_str_list = HattrickMatch.matchDictListToMatchStrList(match_dict_list)

print("\ncat match_str_List\n")
for str_line in match_str_list:
    print(str_line)

In [ ]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

In [ ]:
for line in match_str_list:
    with open (folder + '/match.txt', 'a') as f:
        f.write(line + "\n")

In [ ]:
# From match.txt to match table

In [ ]:
import psycopg2

In [ ]:
# def dropMatchTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """DROP TABLE IF EXISTS match ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
#dropMatchTable()

In [ ]:
# def createMatchTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """
# CREATE TABLE match (
#     Date DATE,          --01
#     Po   VARCHAR(5),    --02 Position
#     Num  INT,           --03 Number
#     RT   NUMERIC(2, 1), --04 Rating
#     sMin INT,           --05 Start Min
#     eMin INT            --06 End Min
# ) ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
# createMatchTable()

In [ ]:
def selectCountPlayer():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = "SELECT COUNT(*) FROM match" 
        
        cur.execute(sql)
        row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        print(sql)
    finally:
        if conn is not None:
            conn.close()
    return row[0]

In [ ]:
selectCountPlayer()

In [ ]:
def insertMatchTable(time, row):
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """
INSERT INTO match ( 
    SELECT 
        to_date(%s, 'YYYY-MM-DD'), --01 
        %s, %s, %s, %s, %s         --02 03 04 05 06
) ;"""
        cur.execute(sql, ( time,
                           row[0], row[1], row[2], row[3], row[4] ) )
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
        print(sql)
        print(row)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
import csv
with open(folder + '/match.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if(re.match( '^[0-9]{1,2}', row[1] ) ) :
            insertMatchTable( folder.replace('/','-'), row )

In [ ]:
selectCountPlayer()

In [ ]:
def updateRatingOfMatchTable():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """
update match 
    set rt = player.rt 
from player   
where 
    match.date = player.date 
    and player.date = player.last 
    and match.po = player.po 
    and match.num = player.num 
    and match.rt != player.rt
; """
        cur.execute(sql)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
        print(sql)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
updateRatingOfMatchTable()

In [ ]:
def getTupleListFromMatch(query):
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        
        #print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
query = ""
query += "\n" + "SELECT "
query += "\n" + "    po, num, rt :: VARCHAR(5), smin, emin "
query += "\n" + "FROM match "
query += "\n" + "WHERE "
query += "\n" + "    date ='" + folder.replace('/','-') + "'"
query += "\n" + "ORDER BY "
query += "\n" + "    CASE "
query += "\n" + "        WHEN po = 'KP' THEN 1"
query += "\n" + "        WHEN po = 'WB' THEN 2"
query += "\n" + "        WHEN po = 'CD' THEN 3"
query += "\n" + "        WHEN po = 'W'  THEN 4"
query += "\n" + "        WHEN po = 'IM' THEN 5"
query += "\n" + "        WHEN po = 'FW' THEN 6"
query += "\n" + "        ELSE 7 "
query += "\n" + "    END, "
query += "\n" + "    emin asc, smin desc"

tupleList = getTupleListFromMatch(query)
for tuple in tupleList:
    print(tuple[0], tuple[1], tuple[2], tuple[3], tuple[4], sep='\t')

In [ ]:
HattrickFile.check_and_delete_file(folder + '/match.txt')

In [ ]:
with open ( folder + '/match.txt', 'a' ) as f:
    f.write("po,num,rt,sMin,eMin\n")
    f.write( '\n'.join('%s,%s,%s,%s,%s' % myTuple for myTuple in tupleList) + '\n' )

In [ ]:
def getKP():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        
        query =  """SELECT * """                                                        + "\n"
        query += """FROM crosstab( """                                                  + "\n"
        query += """    'SELECT """                                                     + "\n"
        query += """        date, """                                                   + "\n"
        query += """        num, """                                                    + "\n"
        query += """        CASE """                                                    + "\n"
        query += """            WHEN length(time_rt)=5 THEN ''  ''||''''||time_rt """   + "\n"
        query += """            WHEN length(time_rt)=6 THEN '' ''||''''||time_rt """    + "\n"
        query += """            WHEN length(time_rt)=7 THEN time_rt """                 + "\n"
        query += """            ELSE ''error'' """                                      + "\n"
        query += """        END AS time_rt """                                          + "\n"
        query += """    FROM ( """                                                      + "\n"
        query += """        SELECT """                                                  + "\n"
        query += """            date :: VARCHAR(10), """                                + "\n"
        query += """            num :: VARCHAR(2), """                                  + "\n"
        query += """            emin-smin||''/''||rt :: VARCHAR(7) as time_rt """       + "\n"
        query += """        FROM match """                                              + "\n"
        query += """        WHERE """                                                   + "\n"
        query += """            po = ''KP'' and current_date - 30 < date """            + "\n"
        query += """        ORDER BY """                                                + "\n"
        query += """            date, num """                                           + "\n"
        query += """    ) AS a', """                                                    + "\n"
        query += """    'SELECT m from unnest(''{1,2}''::int[]) m' """                  + "\n"
        query += """) """                                                               + "\n"
        query += """AS ( """                                                            + "\n"
        query += """    "date num min/rt" VARCHAR(10), """                              + "\n"
        query += """    "   01  " VARCHAR(8), """                                       + "\n"
        query += """    "   02  " VARCHAR(8) """                                        + "\n"
        query += """) """                                                               + "\n"
        query += """ORDER BY 1 """                                                      + "\n"

        #print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
def xstr(s):
    if s is None:
        return '       '
    else :
        if len(s) == 6:
            s = ' ' + s
        if len(s) == 5:
            s = '  ' + s
    return str(s)

In [ ]:
tupleList = getKP()
print('date num min/rt |    01   |    02    ')
print('----------------+---------+----------')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ', xstr(tuple[1]), xstr(tuple[2]), sep=" | ")

In [ ]:
def getWB():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        
        query =  """SELECT * """                                                        + "\n"
        query += """FROM crosstab( """                                                  + "\n"
        query += """    'SELECT """                                                     + "\n"
        query += """        date, """                                                   + "\n"
        query += """        num, """                                                    + "\n"
        query += """        CASE """                                                    + "\n"
        query += """            WHEN length(time_rt)=5 THEN ''  ''||''''||time_rt """   + "\n"
        query += """            WHEN length(time_rt)=6 THEN '' ''||''''||time_rt """    + "\n"
        query += """            WHEN length(time_rt)=7 THEN time_rt """                 + "\n"
        query += """            ELSE ''error'' """                                      + "\n"
        query += """        END AS time_rt """                                          + "\n"
        query += """    FROM ( """                                                      + "\n"
        query += """        SELECT """                                                  + "\n"
        query += """            date :: VARCHAR(10), """                                + "\n"
        query += """            num :: VARCHAR(2), """                                  + "\n"
        query += """            emin-smin||''/''||rt :: VARCHAR(7) as time_rt """       + "\n"
        query += """        FROM match """                                              + "\n"
        query += """        WHERE """                                                   + "\n"
        query += """            po = ''WB'' and current_date - 30 < date """            + "\n"
        query += """        ORDER BY """                                                + "\n"
        query += """            date, num """                                           + "\n"
        query += """    ) AS a', """                                                    + "\n"
        query += """    'SELECT m from unnest(''{44,46,54,58}''::int[]) m' """          + "\n"
        query += """) """                                                               + "\n"
        query += """AS ( """                                                            + "\n"
        query += """    "date num min/rt" VARCHAR(10), """                              + "\n"
        query += """    "   44  " VARCHAR(8), """                                       + "\n"
        query += """    "   46  " VARCHAR(8), """                                       + "\n"
        query += """    "   54  " VARCHAR(8), """                                       + "\n"
        query += """    "   58  " VARCHAR(8) """                                        + "\n"
        query += """) """                                                               + "\n"
        query += """ORDER BY 1 """                                                      + "\n"

#         print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print(query +" \n")
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
tupleList = getWB()
print('date num min/rt |    44   |    46   |    54   |    58   ')
print('----------------+---------+---------+---------+---------')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ', xstr(tuple[1]), xstr(tuple[2]), xstr(tuple[3]), xstr(tuple[4]), sep=" | ")

In [ ]:
def getCD():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()

        query =  """SELECT * """                                                        + "\n"
        query += """FROM crosstab( """                                                  + "\n"
        query += """    'SELECT """                                                     + "\n" 
        query += """        date, """                                                   + "\n"
        query += """        num, """                                                    + "\n"
        query += """        CASE """                                                    + "\n"
        query += """            WHEN length(time_rt)=5 THEN ''  ''||''''||time_rt """   + "\n"
        query += """            WHEN length(time_rt)=6 THEN '' ''||''''||time_rt """    + "\n"
        query += """            WHEN length(time_rt)=7 THEN time_rt """                 + "\n"
        query += """            ELSE ''error'' """                                      + "\n"
        query += """        END AS time_rt """                                          + "\n"
        query += """    FROM ( """                                                      + "\n"
        query += """        SELECT """                                                  + "\n"
        query += """            date :: VARCHAR(10), """                                + "\n"
        query += """            num :: VARCHAR(2), """                                  + "\n"
        query += """            emin-smin||''/''||rt :: VARCHAR(7) as time_rt """       + "\n" 
        query += """        FROM match """                                              + "\n" 
        query += """        WHERE """                                                   + "\n" 
        query += """            po = ''CD'' and current_date - 30 < date """            + "\n" 
        query += """        ORDER BY """                                                + "\n"
        query += """            date, num """                                           + "\n"
        query += """    ) AS a', """                                                    + "\n"
        query += """    'SELECT m from unnest(''{33,43,48,59,61,64}''::int[]) m' """    + "\n" 
        query += """) """                                                               + "\n"
        query += """AS ( """                                                            + "\n"
        query += """    "date num min/rt" VARCHAR(10), """                              + "\n" 
        query += """    "   33  " VARCHAR(8), """                                       + "\n"
        query += """    "   43  " VARCHAR(8), """                                       + "\n"
        query += """    "   48  " VARCHAR(8), """                                       + "\n"
        query += """    "   59  " VARCHAR(8), """                                       + "\n"
        query += """    "   61  " VARCHAR(8), """                                       + "\n"
        query += """    "   64  " VARCHAR(8) """                                        + "\n"
        query += """) """                                                               + "\n"
        query += """ORDER BY 1 """                                                      + "\n"

#         print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
tupleList = getCD()
print('date num min/rt |    33   |    43   |    48   |    59   |    61   |    64   |')
print('----------------+---------+---------+---------+---------+---------+---------+')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ', xstr(tuple[1]), xstr(tuple[2]), xstr(tuple[3]), xstr(tuple[4]), xstr(tuple[5]), xstr(tuple[6]),sep=" | ")

In [ ]:
def getW():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        
        query =  """SELECT * """                                                        + "\n"
        query += """FROM crosstab( """                                                  + "\n"
        query += """    'SELECT """                                                     + "\n"
        query += """        date, """                                                   + "\n"
        query += """        num, """                                                    + "\n"
        query += """        CASE """                                                    + "\n"
        query += """            WHEN length(time_rt)=5 THEN ''  ''||''''||time_rt """   + "\n"
        query += """            WHEN length(time_rt)=6 THEN '' ''||''''||time_rt """    + "\n"
        query += """            WHEN length(time_rt)=7 THEN time_rt """                 + "\n"
        query += """            ELSE ''error'' """                                      + "\n"
        query += """        END AS time_rt """                                          + "\n"
        query += """    FROM ( """                                                      + "\n"
        query += """        SELECT """                                                  + "\n"
        query += """            date :: VARCHAR(10), """                                + "\n"
        query += """            num :: VARCHAR(2), """                                  + "\n"
        query += """            emin-smin||''/''||rt :: VARCHAR(7) as time_rt """       + "\n"
        query += """        FROM match """                                              + "\n"
        query += """        WHERE """                                                   + "\n"
        query += """            po = ''W'' and current_date - 30 < date """             + "\n"
        query += """        ORDER BY """                                                + "\n"
        query += """            date, num """                                           + "\n"
        query += """    ) AS a', """                                                    + "\n"
        query += """    'SELECT m from unnest(''{3,7,13,15,63}''::int[]) m' """         + "\n"
        query += """) """                                                               + "\n"
        query += """AS ( """                                                            + "\n"
        query += """    "date num min/rt" VARCHAR(10), """                              + "\n"
        query += """    "   03  " VARCHAR(8), """                                       + "\n"
        query += """    "   07  " VARCHAR(8), """                                       + "\n"
        query += """    "   13  " VARCHAR(8), """                                       + "\n"
        query += """    "   15  " VARCHAR(8), """                                       + "\n"
        query += """    "   63  " VARCHAR(8) """                                        + "\n"
        query += """) """                                                               + "\n"
        query += """ORDER BY 1 """                                                      + "\n"
        
#         print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
tupleList = getW()
print('date num min/rt |    03   |    07   |    13   |    15   |    63   |')
print('----------------+---------+---------+---------+---------+---------+')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ', xstr(tuple[1]), xstr(tuple[2]), xstr(tuple[3]), xstr(tuple[4]), xstr(tuple[5]), sep=" | ")

In [ ]:
def getIM():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        
        query =  """SELECT * """                                                        + "\n"
        query += """FROM crosstab( """                                                  + "\n"
        query += """    'SELECT """                                                     + "\n"
        query += """        date, """                                                   + "\n"
        query += """        num, """                                                    + "\n"
        query += """        CASE """                                                    + "\n"
        query += """            WHEN length(time_rt)=5 THEN ''  ''||''''||time_rt """   + "\n"
        query += """            WHEN length(time_rt)=6 THEN '' ''||''''||time_rt """    + "\n"
        query += """            WHEN length(time_rt)=7 THEN time_rt """                 + "\n"
        query += """            ELSE ''error'' """                                      + "\n"
        query += """        END AS time_rt """                                          + "\n"
        query += """    FROM ( """                                                      + "\n"
        query += """        SELECT """                                                  + "\n"
        query += """            date :: VARCHAR(10), """                                + "\n"
        query += """            num :: VARCHAR(2), """                                  + "\n"
        query += """            emin-smin||''/''||rt :: VARCHAR(7) as time_rt """       + "\n"
        query += """        FROM match """                                              + "\n"
        query += """        WHERE """                                                   + "\n"
        query += """            po = ''IM'' and current_date - 30 < date """            + "\n"
        query += """        ORDER BY """                                                + "\n"
        query += """            date, num """                                           + "\n"
        query += """    ) AS a', """                                                    + "\n"
        query += """    'SELECT m from unnest(''{9,10,11,12,31}''::int[]) m' """        + "\n"
        query += """) """                                                               + "\n"
        query += """AS ( """                                                            + "\n"
        query += """    "date num min/rt" VARCHAR(10), """                              + "\n"
        query += """    "   09  " VARCHAR(8), """                                       + "\n"
        query += """    "   10  " VARCHAR(8), """                                       + "\n"
        query += """    "   11  " VARCHAR(8), """                                       + "\n"
        query += """    "   12  " VARCHAR(8), """                                       + "\n"
        query += """    "   31  " VARCHAR(8) """                                        + "\n"
        query += """) """                                                               + "\n"
        query += """ORDER BY 1 """                                                      + "\n"

#         print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
tupleList = getIM()
print('date num min/rt |    09   |    10   |    11   |    12   |    31   |')
print('----------------+---------+---------+---------+---------+---------+')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ', xstr(tuple[1]), xstr(tuple[2]), xstr(tuple[3]), xstr(tuple[4]), xstr(tuple[5]), sep=" | ")

In [ ]:
def getFW():
    outputList = list();
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        
        query =  """SELECT * """                                                        + "\n"
        query += """FROM crosstab( """                                                  + "\n"
        query += """    'SELECT """                                                     + "\n"
        query += """        date, """                                                   + "\n"
        query += """        num, """                                                    + "\n"
        query += """        CASE """                                                    + "\n"
        query += """            WHEN length(time_rt)=5 THEN ''  ''||''''||time_rt """   + "\n"
        query += """            WHEN length(time_rt)=6 THEN '' ''||''''||time_rt """    + "\n"
        query += """            WHEN length(time_rt)=7 THEN time_rt """                 + "\n"
        query += """            ELSE ''error'' """                                      + "\n"
        query += """        END AS time_rt """                                          + "\n"
        query += """    FROM ( """                                                      + "\n"
        query += """        SELECT """                                                  + "\n"
        query += """            date :: VARCHAR(10), """                                + "\n"
        query += """            num :: VARCHAR(2), """                                  + "\n"
        query += """            emin-smin||''/''||rt :: VARCHAR(7) as time_rt """       + "\n"
        query += """        FROM match """                                              + "\n"
        query += """        WHERE """                                                   + "\n"
        query += """            po = ''FW'' and current_date - 30 < date """            + "\n"
        query += """        ORDER BY """                                                + "\n"
        query += """            date, num """                                           + "\n"
        query += """    ) AS a', """                                                    + "\n"
        query += """    'SELECT m from unnest(''{16,17,18,60}''::int[]) m' """          + "\n"
        query += """) """                                                               + "\n"
        query += """AS ( """                                                            + "\n"
        query += """    "date num min/rt" VARCHAR(10), """                              + "\n"
        query += """    "   16  " VARCHAR(8), """                                       + "\n"
        query += """    "   17  " VARCHAR(8), """                                       + "\n"
        query += """    "   18  " VARCHAR(8), """                                       + "\n"
        query += """    "   60  " VARCHAR(8) """                                        + "\n"
        query += """) """                                                               + "\n"
        query += """ORDER BY 1 """                                                      + "\n"

#         print(query + "\n")
        cur.execute(query)
        row = cur.fetchone()
        while row is not None:
            #print(row)
            outputList.append(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error Happend")
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return outputList

In [ ]:
tupleList = getFW()
print('date num min/rt |    16   |    17   |    18   |    60   |')
print('----------------+---------+---------+---------+---------+')
for tuple in tupleList:
    #print( len(tuple) )
    print(tuple[0] + '     ',xstr(tuple[1]), xstr(tuple[2]), xstr(tuple[3]), xstr(tuple[4]), sep=" | ")